In [1]:
import optuna
import os
from pathlib import Path
import kaggle
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import HistGradientBoostingClassifier


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
else:
    path = Path('../input/playground-series-s3e23')
    orig_path = Path('../input/software-defect-prediction')
    if not path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)
    if not orig_path.exists():
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

SEED = 42
SPLITS = 5
VERSION = 2
OUTPUT = f'hist_grad_boost_clf_submisson_v{VERSION}.csv'
N_TRIALS = 30

np.random.seed(SEED)

/home/strawhatdragon100/miniconda3/envs/kaggle/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(path/'train.csv', index_col = 'id')
test = pd.read_csv(path/'test.csv', index_col = 'id')
orig_train = pd.read_csv(orig_path/'jm1.csv')

In [3]:
for object_features in list(orig_train.loc[:, orig_train.dtypes == 'O']):
    orig_train[object_features] = orig_train[object_features].replace({'?' : np.nan}).astype('float64')

In [4]:
X = pd.concat([train, orig_train])
y = X.pop('defects')

In [5]:
partial_pipeline = make_pipeline(SimpleImputer(), FunctionTransformer(func=np.log1p, validate=False), PowerTransformer())

In [6]:
def objective(trial):
    params = {
        'random_state': trial.suggest_categorical('random_state', [SEED]),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'l2_regularization': trial.suggest_int('l2_regularization', 0, 100),
        'max_iter': trial.suggest_int('max_iter', 10, 10_000),
        'max_leaf_nodes': trial.suggest_int('max_leaf_nodes', 2, 500),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 300),
        'max_bins': trial.suggest_int('max_bins', 32, 255),
        'class_weight': trial.suggest_categorical('class_weight', [None, 'balanced'])
    }
    pipeline = make_pipeline(partial_pipeline, HistGradientBoostingClassifier(**params))
    scores = cross_val_score(pipeline, X, y, cv=SPLITS, scoring='roc_auc', n_jobs=-1)
    return scores.mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)

[I 2023-10-16 14:02:22,834] A new study created in memory with name: no-name-461c84c4-b511-42bf-b2a2-f2815ad108c6
Best trial: 0. Best value: 0.783936:   3%|█████▍                                                                                                                                                              | 1/30 [00:24<11:53, 24.59s/it]

[I 2023-10-16 14:02:47,420] Trial 0 finished with value: 0.7839357596386731 and parameters: {'random_state': 42, 'learning_rate': 0.0162891722738249, 'l2_regularization': 72, 'max_iter': 5659, 'max_leaf_nodes': 333, 'min_samples_leaf': 16, 'max_bins': 147, 'class_weight': 'balanced'}. Best is trial 0 with value: 0.7839357596386731.


Best trial: 1. Best value: 0.784494:   7%|██████████▉                                                                                                                                                         | 2/30 [00:44<10:13, 21.91s/it]

[I 2023-10-16 14:03:07,455] Trial 1 finished with value: 0.784493803378133 and parameters: {'random_state': 42, 'learning_rate': 0.022172329769311373, 'l2_regularization': 63, 'max_iter': 4526, 'max_leaf_nodes': 486, 'min_samples_leaf': 129, 'max_bins': 206, 'class_weight': None}. Best is trial 1 with value: 0.784493803378133.


Best trial: 1. Best value: 0.784494:  10%|████████████████▍                                                                                                                                                   | 3/30 [01:09<10:32, 23.43s/it]

...

[I 2023-10-16 14:06:56,976] Trial 27 finished with value: 0.7859397829690407 and parameters: {'random_state': 42, 'learning_rate': 0.07816505625944993, 'l2_regularization': 66, 'max_iter': 3718, 'max_leaf_nodes': 10, 'min_samples_leaf': 81, 'max_bins': 100, 'class_weight': None}. Best is trial 24 with value: 0.7861680221392143.


Best trial: 24. Best value: 0.786168:  97%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 29/30 [04:41<00:06,  6.41s/it]

[I 2023-10-16 14:07:04,750] Trial 28 finished with value: 0.7852070919804187 and parameters: {'random_state': 42, 'learning_rate': 0.047327583575388604, 'l2_regularization': 77, 'max_iter': 6470, 'max_leaf_nodes': 82, 'min_samples_leaf': 118, 'max_bins': 72, 'class_weight': None}. Best is trial 24 with value: 0.7861680221392143.


Best trial: 24. Best value: 0.786168: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [04:50<00:00,  9.70s/it]

[I 2023-10-16 14:07:13,725] Trial 29 finished with value: 0.7834244287610251 and parameters: {'random_state': 42, 'learning_rate': 0.06221232924690536, 'l2_regularization': 56, 'max_iter': 5724, 'max_leaf_nodes': 295, 'min_samples_leaf': 13, 'max_bins': 146, 'class_weight': 'balanced'}. Best is trial 24 with value: 0.7861680221392143.


In [7]:
study.best_params, study.best_value

({'random_state': 42,
  'learning_rate': 0.07765831939737257,
  'l2_regularization': 60,
  'max_iter': 2456,
  'max_leaf_nodes': 8,
  'min_samples_leaf': 98,
  'max_bins': 45,
  'class_weight': None},
 0.7861680221392143)

In [8]:
pipeline = make_pipeline(partial_pipeline, HistGradientBoostingClassifier(**study.best_params))
pipeline.fit(X, y)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                 ('functiontransformer',
                                  FunctionTransformer(func=<ufunc 'log1p'>)),
                                 ('powertransformer', PowerTransformer())])),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier(l2_regularization=60,
                                                learning_rate=0.07765831939737257,
                                                max_bins=45, max_iter=2456,
                                                max_leaf_nodes=8,
                                                min_samples_leaf=98,
                                                random_state=42))])

In [9]:
submission = test.copy()
submission['defects'] = pipeline.predict_proba(submission)[:, 1]
submission.defects.to_csv(OUTPUT)